# Censo Escolar da Educação Básica

## Configuração do ambiente

In [0]:
spark.conf.set(
  "fs.azure.account.key.m03storage.blob.core.windows.net",
  "3Ts0JpdSfDvYx8JOerU9CrExx1Ankj3W1q5J3Cscu2hXlQR90cpMlVy6p2mBB0NpKndKWixIcp1poqcFrAJ+oQ==")

In [0]:
blobPath = 'wasbs://m03container@m03storage.blob.core.windows.net/'
bronzePath = 'mnt/bronze/censo_escolar/'
silverPath = 'mnt/silver/censo_escolar/'
goldPath = 'mnt/gold/censo_escolar/'

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

## Download das Bases de Dados

In [0]:
%sh
echo 'Iniciando download do Censo Escolar 2017...'
wget https://download.inep.gov.br/microdados/micro_censo_escolar_2017.zip -O /dbfs/mnt/projetom03/censo_escolar_2017.zip

Iniciando download do Censo Escolar 2017...
--2021-03-25 15:04:42-- https://download.inep.gov.br/microdados/micro_censo_escolar_2017.zip
Resolving download.inep.gov.br (download.inep.gov.br)... 200.130.24.15
Connecting to download.inep.gov.br (download.inep.gov.br)|200.130.24.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1750578112 (1.6G) [application/zip]
Saving to: ‘/dbfs/mnt/projetom03/censo_escolar_2017.zip’

 0K .......... .......... .......... .......... .......... 0% 1.15M 24m15s
 50K .......... .......... .......... .......... .......... 0% 2.17M 18m32s
 100K .......... .......... .......... .......... .......... 0% 2.25M 16m28s
 150K .......... .......... .......... .......... .......... 0% 2.32M 15m21s
 200K .......... .......... .......... .......... .......... 0% 12.9M 12m42s
 250K .......... .......... .......... .......... .......... 0% 2.76M 12m16s
 300K .......... .......... .......... .......... .......... 0% 2.88M 11m54s
 350K .......... .......... .......... .......... .......... 0% 8.42M 10m49s
 400K .......... .......... .......... .......... .......... 0% 9.64M 9m56s
 450K .......... .......... .......... .......... .......... 0% 520K 14m25s
 500K .......... .......... .......... .......... .......... 0% 552K 17m48s
 550K .......... .......... .......... .......... .......... 0% 10.2M 16m33s
 600K .......... .......... .......... .......... .......... 0% 19.4M 15m23s
 650K .......... .......... .......... .......... .......... 0% 2.99M 14m57s
 700K .......... .......... .......... .......... .......... 0% 8.70M 14m10s
 750K .......... .......... .......... .......... .......... 0% 3.14M 13m50s
 800K .......... .......... .......... .......... .......... 0% 11.0M 13m10s
 850K .......... .......... .......... .......... .......... 0% 15.2M 12m32s
 900K .......... .......... .......... .......... .......... 0% 2.47M 12m28s
 950K .......... .......... .......... .......... .......... 0% 12.6M 11m57s
 1000K .......... .......... .......... .......... .......... 0% 2.95M 11m50s
 1050K .......... .......... .......... .......... .......... 0% 16.5M 11m22s
 1100K .......... .......... .......... .......... .......... 0% 21.7M 10m56s
 1150K .......... .......... .......... .......... .......... 0% 1.86M 11m6s
 1200K .......... .......... .......... .......... .......... 0% 10.4M 10m46s
 1250K .......... .......... .......... .......... .......... 0% 4.09M 10m37s
 1300K .......... .......... .......... .......... .......... 0% 13.4M 10m18s
 1350K .......... .......... .......... .......... .......... 0% 2.95M 10m16s
 1400K .......... .......... .......... .......... .......... 0% 15.9M 9m58s
 1450K .......... .......... .......... .......... .......... 0% 2.75M 9m58s
 1500K .......... .......... .......... .......... .......... 0% 11.8M 9m44s
 1550K .......... .......... .......... .......... .......... 0% 12.0M 9m30s
 1600K .......... .......... .......... .......... .......... 0% 2.81M 9m30s
 1650K .......... .......... .......... .......... .......... 0% 4.92M 9m24s
 1700K .......... .......... .......... .......... .......... 0% 4.71M 9m18s
 1750K .......... .......... .......... .......... .......... 0% 13.5M 9m6s
 1800K .......... .......... .......... .......... .......... 0% 2.92M 9m6s
 1850K .......... .......... .......... .......... .......... 0% 8.90M 8m57s
 1900K .......... .......... .......... .......... .......... 0% 15.6M 8m46s
 1950K .......... .......... .......... .......... .......... 0% 2.39M 8m50s
 2000K .......... .......... .......... .......... .......... 0% 19.3M 8m39s
 2050K .......... .......... .......... .......... .......... 0% 3.22M 8m39s
 2100K .......... .......... .......... .......... .......... 0% 11.5M 8m30s
 2150K .......... .......... .......... .......... .......... 0% 17.0M 8m21s
 2200K .......... .......... .......... .......... .......... 0% 2.83M 8m23s
 2250K .......... .......... .......... .......... .......... 0% 10.3M 8m16s
 2300K ....

In [0]:
%sh
echo 'Iniciando download do Censo Escolar 2018...'
wget https://download.inep.gov.br/microdados/microdados_educacao_basica_2018.zip -O /dbfs/mnt/projetom03/censo_escolar_2018.zip

Iniciando download do Censo Escolar 2018...
--2021-03-25 14:56:44-- https://download.inep.gov.br/microdados/microdados_educacao_basica_2018.zip
Resolving download.inep.gov.br (download.inep.gov.br)... 200.130.24.15
Connecting to download.inep.gov.br (download.inep.gov.br)|200.130.24.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2692342575 (2.5G) [application/zip]
Saving to: ‘/dbfs/mnt/projetom03/censo_escolar_2018.zip’

 0K .......... .......... .......... .......... .......... 0% 1.21M 35m26s
 50K .......... .......... .......... .......... .......... 0% 2.16M 27m37s
 100K .......... .......... .......... .......... .......... 0% 2.50M 24m7s
 150K .......... .......... .......... .......... .......... 0% 2.73M 22m0s
 200K .......... .......... .......... .......... .......... 0% 15.6M 18m9s
 250K .......... .......... .......... .......... .......... 0% 2.71M 17m45s
 300K .......... .......... .......... .......... .......... 0% 14.5M 15m38s
 350K .......... .......... .......... .......... .......... 0% 3.15M 15m23s
 400K .......... .......... .......... .......... .......... 0% 12.4M 14m3s
 450K .......... .......... .......... .......... .......... 0% 3.21M 13m59s
 500K .......... .......... .......... .......... .......... 0% 17.3M 12m56s
 550K .......... .......... .......... .......... .......... 0% 16.3M 12m5s
 600K .......... .......... .......... .......... .......... 0% 2.83M 12m19s
 650K .......... .......... .......... .......... .......... 0% 14.3M 11m39s
 700K .......... .......... .......... .......... .......... 0% 3.21M 11m45s
 750K .......... .......... .......... .......... .......... 0% 13.6M 11m13s
 800K .......... .......... .......... .......... .......... 0% 15.5M 10m43s
 850K .......... .......... .......... .......... .......... 0% 3.05M 10m54s
 900K .......... .......... .......... .......... .......... 0% 18.0M 10m27s
 950K .......... .......... .......... .......... .......... 0% 2.98M 10m39s
 1000K .......... .......... .......... .......... .......... 0% 11.9M 10m19s
 1050K .......... .......... .......... .......... .......... 0% 23.0M 9m56s
 1100K .......... .......... .......... .......... .......... 0% 3.06M 10m6s
 1150K .......... .......... .......... .......... .......... 0% 13.4M 9m49s
 1200K .......... .......... .......... .......... .......... 0% 2.99M 10m0s
 1250K .......... .......... .......... .......... .......... 0% 13.4M 9m44s
 1300K .......... .......... .......... .......... .......... 0% 16.1M 9m28s
 1350K .......... .......... .......... .......... .......... 0% 3.12M 9m37s
 1400K .......... .......... .......... .......... .......... 0% 16.1M 9m23s
 1450K .......... .......... .......... .......... .......... 0% 3.05M 9m32s
 1500K .......... .......... .......... .......... .......... 0% 13.7M 9m20s
 1550K .......... .......... .......... .......... .......... 0% 15.6M 9m7s
 1600K .......... .......... .......... .......... .......... 0% 2.52M 9m22s
 1650K .......... .......... .......... .......... .......... 0% 9.40M 9m13s
 1700K .......... .......... .......... .......... .......... 0% 3.92M 9m16s
 1750K .......... .......... .......... .......... .......... 0% 6.79M 9m11s
 1800K .......... .......... .......... .......... .......... 0% 4.27M 9m12s
 1850K .......... .......... .......... .......... .......... 0% 7.80M 9m6s
 1900K .......... .......... .......... .......... .......... 0% 12.1M 8m58s
 1950K .......... .......... .......... .......... .......... 0% 3.29M 9m4s
 2000K .......... .......... .......... .......... .......... 0% 9.66M 8m57s
 2050K .......... .......... .......... .......... .......... 0% 4.20M 8m59s
 2100K .......... .......... .......... .......... .......... 0% 5.45M 8m57s
 2150K .......... .......... .......... .......... .......... 0% 4.57M 8m58s
 2200K .......... .......... .......... .......... .......... 0% 7.09M 8m54s
 2250K .......... .......... .......... .......... .......... 0% 10.4M 8m48s
 2300K ......

In [0]:
%sh 
echo 'Iniciando download do Censo Escolar 2019...'
wget https://download.inep.gov.br/microdados/microdados_educacao_basica_2019.zip -O /dbfs/mnt/projetom03/censo_escolar_2019.zip

Iniciando download do Censo Escolar 2019...
--2021-03-25 14:40:04-- https://download.inep.gov.br/microdados/microdados_educacao_basica_2019.zip
Resolving download.inep.gov.br (download.inep.gov.br)... 200.130.24.15
Connecting to download.inep.gov.br (download.inep.gov.br)|200.130.24.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2804740605 (2.6G) [application/zip]
Saving to: ‘/dbfs/mnt/projetom03/censo_escolar_2019.zip’

 0K .......... .......... .......... .......... .......... 0% 1.19M 37m33s
 50K .......... .......... .......... .......... .......... 0% 1.91M 30m28s
 100K .......... .......... .......... .......... .......... 0% 2.07M 27m29s
 150K .......... .......... .......... .......... .......... 0% 2.58M 24m56s
 200K .......... .......... .......... .......... .......... 0% 12.6M 20m39s
 250K .......... .......... .......... .......... .......... 0% 4.07M 19m2s
 300K .......... .......... .......... .......... .......... 0% 2.30M 19m5s
 350K .......... .......... .......... .......... .......... 0% 13.4M 17m7s
 400K .......... .......... .......... .......... .......... 0% 16.2M 15m31s
 450K .......... .......... .......... .......... .......... 0% 3.42M 15m16s
 500K .......... .......... .......... .......... .......... 0% 3.75M 14m58s
 550K .......... .......... .......... .......... .......... 0% 6.19M 14m19s
 600K .......... .......... .......... .......... .......... 0% 4.11M 14m3s
 650K .......... .......... .......... .......... .......... 0% 6.45M 13m32s
 700K .......... .......... .......... .......... .......... 0% 7.18M 13m3s
 750K .......... .......... .......... .......... .......... 0% 3.80M 12m58s
 800K .......... .......... .......... .......... .......... 0% 6.30M 12m37s
 850K .......... .......... .......... .......... .......... 0% 5.38M 12m23s
 900K .......... .......... .......... .......... .......... 0% 4.53M 12m15s
 950K .......... .......... .......... .......... .......... 0% 5.55M 12m2s
 1000K .......... .......... .......... .......... .......... 0% 6.01M 11m49s
 1050K .......... .......... .......... .......... .......... 0% 6.89M 11m34s
 1100K .......... .......... .......... .......... .......... 0% 4.41M 11m30s
 1150K .......... .......... .......... .......... .......... 0% 6.08M 11m20s
 1200K .......... .......... .......... .......... .......... 0% 5.34M 11m13s
 1250K .......... .......... .......... .......... .......... 0% 4.05M 11m12s
 1300K .......... .......... .......... .......... .......... 0% 6.50M 11m3s
 1350K .......... .......... .......... .......... .......... 0% 3.88M 11m3s
 1400K .......... .......... .......... .......... .......... 0% 14.5M 10m47s
 1450K .......... .......... .......... .......... .......... 0% 5.73M 10m41s
 1500K .......... .......... .......... .......... .......... 0% 4.08M 10m41s
 1550K .......... .......... .......... .......... .......... 0% 6.80M 10m34s
 1600K .......... .......... .......... .......... .......... 0% 3.88M 10m35s
 1650K .......... .......... .......... .......... .......... 0% 6.55M 10m29s
 1700K .......... .......... .......... .......... .......... 0% 3.94M 10m30s
 1750K .......... .......... .......... .......... .......... 0% 13.5M 10m18s
 1800K .......... .......... .......... .......... .......... 0% 3.76M 10m20s
 1850K .......... .......... .......... .......... .......... 0% 7.21M 10m14s
 1900K .......... .......... .......... .......... .......... 0% 4.12M 10m15s
 1950K .......... .......... .......... .......... .......... 0% 3.49M 10m19s
 2000K .......... .......... .......... .......... .......... 0% 8.63M 10m11s
 2050K .......... .......... .......... .......... .......... 0% 5.03M 10m9s
 2100K .......... .......... .......... .......... .......... 0% 11.2M 10m1s
 2150K .......... .......... .......... .......... .......... 0% 3.60M 10m4s
 2200K .......... .......... .......... .......... .......... 0% 8.45M 9m57s
 2250K .......... .......... .......... .......... .......... 0% 4.30M

In [0]:
dbutils.fs.cp(blobPath + "censo_escolar_2019.zip", "file:/tmp/censo_escolar_2019.zip")
dbutils.fs.cp(blobPath + "censo_escolar_2018.zip", "file:/tmp/censo_escolar_2018.zip")
dbutils.fs.cp(blobPath + "censo_escolar_2017.zip", "file:/tmp/censo_escolar_2017.zip")

Out[4]: True

Descompactação dos arquivos **.zip** sem a sua estrutura de diretórios em pastas para cada ano do censo

In [0]:
%sh
unzip -j /tmp/censo_escolar_2019.zip -d /tmp/censo2019/
unzip -j /tmp/censo_escolar_2018.zip -d /tmp/censo2018/
unzip -j /tmp/censo_escolar_2017.zip -d /tmp/censo2017/
unzip -j /tmp/censo2018/'*.zip' -d /tmp/censo2018/
unzip -j /tmp/censo2017/'*.zip' -d /tmp/censo2017/

Archive: /tmp/censo_escolar_2019.zip
 inflating: /tmp/censo2019/Dicionаrio de Dados da EducaЗ╞o Bаsica.xlsx 
 inflating: /tmp/censo2019/Tabelas Auxiliares.xlsx 
 inflating: /tmp/censo2019/Cadastro_Aluno.pdf 
 inflating: /tmp/censo2019/Cadastro_Escola.pdf 
 inflating: /tmp/censo2019/Cadastro_Gestor_Escolar.pdf 
 inflating: /tmp/censo2019/Cadastro_Profissional_Escolar.pdf 
 inflating: /tmp/censo2019/Cadastro_Turma.pdf 
 inflating: /tmp/censo2019/DOCENTES_CO.CSV 
 inflating: /tmp/censo2019/DOCENTES_NORDESTE.CSV 
 inflating: /tmp/censo2019/DOCENTES_NORTE.CSV 
 inflating: /tmp/censo2019/DOCENTES_SUDESTE.CSV 
 inflating: /tmp/censo2019/DOCENTES_SUL.CSV 
 inflating: /tmp/censo2019/ESCOLAS.CSV 
 inflating: /tmp/censo2019/GESTOR.CSV 
 inflating: /tmp/censo2019/MATRICULA_CO.CSV 
 inflating: /tmp/censo2019/MATRICULA_NORDESTE.CSV 
 inflating: /tmp/censo2019/MATRICULA_NORTE.CSV 
 inflating: /tmp/censo2019/MATRICULA_SUDESTE.CSV 
 inflating: /tmp/censo2019/MATRICULA_SUL.CSV 
 inflating: /tmp/censo2019/MD5_microdados_ed_basica_2019.txt 
 inflating: /tmp/censo2019/TURMAS.CSV 
 inflating: /tmp/censo2019/Filtros Censo Ed Basica 2019.pdf 
 inflating: /tmp/censo2019/Leia-me 2019.pdf 
 inflating: /tmp/censo2019/Nota Informativa nз 01.pdf 
 inflating: /tmp/censo2019/Nota Informativa nз 02.pdf 
Archive: /tmp/censo_escolar_2018.zip
 inflating: /tmp/censo2018/Dicionаrio de Dados da EducaЗ╞o Bаsica.xlsx 
 inflating: /tmp/censo2018/Tabelas Auxiliares.xlsx 
 inflating: /tmp/censo2018/Cadastro_Aluno.pdf 
 inflating: /tmp/censo2018/Cadastro_Escola.pdf 
 inflating: /tmp/censo2018/Cadastro_Profissional_Escolar.pdf 
 inflating: /tmp/censo2018/Cadastro_Turma.pdf 
 extracting: /tmp/censo2018/DOCENTES_CO.zip 
 extracting: /tmp/censo2018/DOCENTES_NORDESTE.zip 
 inflating: /tmp/censo2018/DOCENTES_NORTE.zip 
 inflating: /tmp/censo2018/DOCENTES_SUDESTE.zip 
 inflating: /tmp/censo2018/DOCENTES_SUL.zip 
 extracting: /tmp/censo2018/ESCOLAS.zip 
 extracting: /tmp/censo2018/MATRICULA_CO.zip 
 extracting: /tmp/censo2018/MATRICULA_NORDESTE.zip 
 extracting: /tmp/censo2018/MATRICULA_NORTE.zip 
 extracting: /tmp/censo2018/MATRICULA_SUDESTE.zip 
 extracting: /tmp/censo2018/MATRICULA_SUL.zip 
 inflating: /tmp/censo2018/md5_microdados_ed_basica_2018.txt 
 extracting: /tmp/censo2018/TURMAS.zip 
 inflating: /tmp/censo2018/Filtros da EducaЗ╞o Bаsica.pdf 
 inflating: /tmp/censo2018/Leia-me.pdf 
 inflating: /tmp/censo2018/Nota_informativa_versao_01.pdf 
Archive: /tmp/censo_escolar_2017.zip
 inflating: /tmp/censo2017/Dicionаrio de Dados da EducaЗ╞o Bаsica 2017.xlsx 
 inflating: /tmp/censo2017/Tabelas Auxiliares.xlsx 
 inflating: /tmp/censo2017/cadastro_aluno.pdf 
 inflating: /tmp/censo2017/cadastro_escola.pdf 
 inflating: /tmp/censo2017/cadastro_profissional_escolar.pdf 
 inflating: /tmp/censo2017/cadastro_turma.pdf 
 extracting: /tmp/censo2017/DOCENTES_CO.zip 
 extracting: /tmp/censo2017/DOCENTES_NORDESTE.zip 
 inflating: /tmp/censo2017/DOCENTES_NORTE.zip 
 extracting: /tmp/censo2017/DOCENTES_SUDESTE.zip 
 extracting: /tmp/censo2017/DOCENTES_SUL.zip 
 extracting: /tmp/censo2017/ESCOLAS.zip 
 extracting: /tmp/censo2017/MATRICULA_CO.zip 
 extracting: /tmp/censo2017/MATRICULA_NORDESTE.zip 
 extracting: /tmp/censo2017/MATRICULA_NORTE.zip 
 extracting: /tmp/censo2017/MATRICULA_SUDESTE.zip 
 extracting: /tmp/censo2017/MATRICULA_SUL.zip 
 inflating: /tmp/censo2017/md5_microdados_educaЗ╞o_bаsica_2017.txt 
 extracting: /tmp/censo2017/TURMAS.zip 
 inflating: /tmp/censo2017/Filtros Censo Ed Basica 2017.pdf 
 inflating: /tmp/censo2017/Leia-me 2017.pdf 
Archive: /tmp/censo2018/MATRICULA_NORTE.zip
 inflating: /tmp/censo2018/MATRICULA_NORTE.CSV 

Archive: /tmp/censo2018/DOCENTES_SUL.zip
 inflating: /tmp/censo2018/DOCENTES_SUL.CSV 

Archive: /tmp/censo2018/MATRICULA_SUDESTE.zip
 inflating: /tmp/censo2018/MATRICULA_SUDESTE.CSV 

Archive: /tmp/censo2018/MATRICULA_SUL.zip
 inflating: /tmp/censo2018/MATRICULA_SUL.CSV 

Archive: /tmp/censo2018/DOCENTES_CO.zip
 inflating: /tmp/censo2018/DOCENTES_CO.CSV 



Remoção de todos os arquivos que não possuem a extensão **.CSV**

In [0]:
%sh
shopt -s extglob
rm -v /tmp/censo2019/!(*.CSV)
rm -v /tmp/censo2018/!(*.CSV)
rm -v /tmp/censo2017/!(*.CSV)
shopt -u extglob

removed '/tmp/censo2019/Cadastro_Aluno.pdf'
removed '/tmp/censo2019/Cadastro_Escola.pdf'
removed '/tmp/censo2019/Cadastro_Gestor_Escolar.pdf'
removed '/tmp/censo2019/Cadastro_Profissional_Escolar.pdf'
removed '/tmp/censo2019/Cadastro_Turma.pdf'
removed '/tmp/censo2019/Dicionаrio de Dados da EducaЗ╞o Bаsica.xlsx'
removed '/tmp/censo2019/Filtros Censo Ed Basica 2019.pdf'
removed '/tmp/censo2019/Leia-me 2019.pdf'
removed '/tmp/censo2019/MD5_microdados_ed_basica_2019.txt'
removed '/tmp/censo2019/Nota Informativa nз 01.pdf'
removed '/tmp/censo2019/Nota Informativa nз 02.pdf'
removed '/tmp/censo2019/Tabelas Auxiliares.xlsx'
removed '/tmp/censo2018/Cadastro_Aluno.pdf'
removed '/tmp/censo2018/Cadastro_Escola.pdf'
removed '/tmp/censo2018/Cadastro_Profissional_Escolar.pdf'
removed '/tmp/censo2018/Cadastro_Turma.pdf'
removed '/tmp/censo2018/Dicionаrio de Dados da EducaЗ╞o Bаsica.xlsx'
removed '/tmp/censo2018/DOCENTES_CO.zip'
removed '/tmp/censo2018/DOCENTES_NORDESTE.zip'
removed '/tmp/censo2018/DOCENTES_NORTE.zip'
removed '/tmp/censo2018/DOCENTES_SUDESTE.zip'
removed '/tmp/censo2018/DOCENTES_SUL.zip'
removed '/tmp/censo2018/ESCOLAS.zip'
removed '/tmp/censo2018/Filtros da EducaЗ╞o Bаsica.pdf'
removed '/tmp/censo2018/Leia-me.pdf'
removed '/tmp/censo2018/MATRICULA_CO.zip'
removed '/tmp/censo2018/MATRICULA_NORDESTE.zip'
removed '/tmp/censo2018/MATRICULA_NORTE.zip'
removed '/tmp/censo2018/MATRICULA_SUDESTE.zip'
removed '/tmp/censo2018/MATRICULA_SUL.zip'
removed '/tmp/censo2018/md5_microdados_ed_basica_2018.txt'
removed '/tmp/censo2018/Nota_informativa_versao_01.pdf'
removed '/tmp/censo2018/Tabelas Auxiliares.xlsx'
removed '/tmp/censo2018/TURMAS.zip'
removed '/tmp/censo2017/cadastro_aluno.pdf'
removed '/tmp/censo2017/cadastro_escola.pdf'
removed '/tmp/censo2017/cadastro_profissional_escolar.pdf'
removed '/tmp/censo2017/cadastro_turma.pdf'
removed '/tmp/censo2017/Dicionаrio de Dados da EducaЗ╞o Bаsica 2017.xlsx'
removed '/tmp/censo2017/DOCENTES_CO.zip'
removed '/tmp/censo2017/DOCENTES_NORDESTE.zip'
removed '/tmp/censo2017/DOCENTES_NORTE.zip'
removed '/tmp/censo2017/DOCENTES_SUDESTE.zip'
removed '/tmp/censo2017/DOCENTES_SUL.zip'
removed '/tmp/censo2017/ESCOLAS.zip'
removed '/tmp/censo2017/Filtros Censo Ed Basica 2017.pdf'
removed '/tmp/censo2017/Leia-me 2017.pdf'
removed '/tmp/censo2017/MATRICULA_CO.zip'
removed '/tmp/censo2017/MATRICULA_NORDESTE.zip'
removed '/tmp/censo2017/MATRICULA_NORTE.zip'
removed '/tmp/censo2017/MATRICULA_SUDESTE.zip'
removed '/tmp/censo2017/MATRICULA_SUL.zip'
removed '/tmp/censo2017/md5_microdados_educaЗ╞o_bаsica_2017.txt'
removed '/tmp/censo2017/Tabelas Auxiliares.xlsx'
removed '/tmp/censo2017/TURMAS.zip'

## Censo Escolar

### 1. Docentes

#### 1.1. Ingestão dos dados no Delta Lake

In [0]:
# Leitura dos arquivos .CSV de cada ano e região, armazenando-os em dataframes separados.

docentes_co_2017_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2017/DOCENTES_CO.CSV'))
docentes_norte_2017_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2017/DOCENTES_NORTE.CSV'))
docentes_nordeste_2017_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2017/DOCENTES_NORDESTE.CSV'))
docentes_sul_2017_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2017/DOCENTES_SUL.CSV'))
docentes_sudeste_2017_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2017/DOCENTES_SUDESTE.CSV'))

docentes_co_2018_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2018/DOCENTES_CO.CSV'))
docentes_norte_2018_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2018/DOCENTES_NORTE.CSV'))
docentes_nordeste_2018_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2018/DOCENTES_NORDESTE.CSV'))
docentes_sul_2018_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2018/DOCENTES_SUL.CSV'))
docentes_sudeste_2018_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2018/DOCENTES_SUDESTE.CSV'))

docentes_co_2019_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2019/DOCENTES_CO.CSV'))
docentes_norte_2019_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2019/DOCENTES_NORTE.CSV'))
docentes_nordeste_2019_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2019/DOCENTES_NORDESTE.CSV'))
docentes_sul_2019_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2019/DOCENTES_SUL.CSV'))
docentes_sudeste_2019_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2019/DOCENTES_SUDESTE.CSV'))

In [0]:
# Armazenamento dos dataframes na camada Bronze do Delta Lake

docentes_co_2017_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_co_2017')
docentes_norte_2017_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_norte_2017')
docentes_nordeste_2017_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_nordeste_2017')
docentes_sul_2017_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_sul_2017')
docentes_sudeste_2017_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_sudeste_2017')

docentes_co_2018_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_co_2018')
docentes_norte_2018_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_norte_2018')
docentes_nordeste_2018_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_nordeste_2018')
docentes_sul_2018_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_sul_2018')
docentes_sudeste_2018_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_sudeste_2018')

docentes_co_2019_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_co_2019')
docentes_norte_2019_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_norte_2019')
docentes_nordeste_2019_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_nordeste_2019')
docentes_sul_2019_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_sul_2019')
docentes_sudeste_2019_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'docentes_sudeste_2019')

#### 1.2. Transformação dos dados

In [0]:
# Leitura dos dataframes salvos na camada Bronze

docentes_co_2017_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_co_2017')
docentes_norte_2017_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_norte_2017') 
docentes_nordeste_2017_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_nordeste_2017') 
docentes_sul_2017_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_sul_2017') 
docentes_sudeste_2017_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_sudeste_2017')

docentes_co_2018_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_co_2018')
docentes_norte_2018_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_norte_2018') 
docentes_nordeste_2018_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_nordeste_2018') 
docentes_sul_2018_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_sul_2018') 
docentes_sudeste_2018_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_sudeste_2018')

docentes_co_2019_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_co_2019')
docentes_norte_2019_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_norte_2019') 
docentes_nordeste_2019_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_nordeste_2019') 
docentes_sul_2019_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_sul_2019') 
docentes_sudeste_2019_silver = spark.read.format("delta").load(blobPath + bronzePath + 'docentes_sudeste_2019')

In [0]:
# Lista de colunas a serem selecionadas do dataframe original porém com novo título

renamedFields = ['NU_ANO_CENSO as Ano', 
                 'NU_IDADE_REFERENCIA as Idade_Referencia', 
                 'NU_IDADE as Idade',
                 'TP_SEXO as Sexo',
                 'TP_COR_RACA as Cor_Raca', 
                 'TP_NACIONALIDADE as Nacionalidade', 
                 'TP_ZONA_RESIDENCIAL as Zona_Residencial',
                 'TP_ESCOLARIDADE as Escolaridade',
                 'TP_SITUACAO_CURSO_1 as Situacao_Curso_1',
                 'CO_AREA_CURSO_1 as Area_Curso_1',
                 'CO_CURSO_1 as Curso_1',
                 'IN_LICENCIATURA_1 as Licenciatura_Curso_1',
                 'TP_SITUACAO_CURSO_2 as Situacao_Curso_2',
                 'CO_AREA_CURSO_2 as Area_Curso_2',
                 'CO_CURSO_2 as Curso_2',
                 'IN_LICENCIATURA_2 as Licenciatura_Curso_2',
                 'TP_SITUACAO_CURSO_3 as Situacao_Curso_3',
                 'CO_AREA_CURSO_3 as Area_Curso_3',
                 'CO_CURSO_3 as Curso_3',
                 'IN_LICENCIATURA_3 as Licenciatura_Curso_3',
                 'IN_ESPECIALIZACAO as Especializacao',
                 'IN_MESTRADO as Mestrado',
                 'IN_DOUTORADO as Doutorado',
                 'IN_POS_NENHUM as Pos_Graduacao',
                 'TP_TIPO_DOCENTE as Funcao']

In [0]:
# Alteração dos dataframes carregados com base na lista definida anteriormente

docentes_co_2017_silver = docentes_co_2017_silver.selectExpr(renamedFields)
docentes_norte_2017_silver = docentes_norte_2017_silver.selectExpr(renamedFields)
docentes_nordeste_2017_silver = docentes_nordeste_2017_silver.selectExpr(renamedFields)
docentes_sul_2017_silver = docentes_sul_2017_silver.selectExpr(renamedFields)
docentes_sudeste_2017_silver = docentes_sudeste_2017_silver.selectExpr(renamedFields)

docentes_co_2018_silver = docentes_co_2018_silver.selectExpr(renamedFields)
docentes_norte_2018_silver = docentes_norte_2018_silver.selectExpr(renamedFields)
docentes_nordeste_2018_silver = docentes_nordeste_2018_silver.selectExpr(renamedFields)
docentes_sul_2018_silver = docentes_sul_2018_silver.selectExpr(renamedFields)
docentes_sudeste_2018_silver = docentes_sudeste_2018_silver.selectExpr(renamedFields)

docentes_co_2019_silver = docentes_co_2019_silver.selectExpr(renamedFields)
docentes_norte_2019_silver = docentes_norte_2019_silver.selectExpr(renamedFields)
docentes_nordeste_2019_silver = docentes_nordeste_2019_silver.selectExpr(renamedFields)
docentes_sul_2019_silver = docentes_sul_2019_silver.selectExpr(renamedFields)
docentes_sudeste_2019_silver = docentes_sudeste_2019_silver.selectExpr(renamedFields)

In [0]:
# Unificação dos dataframes das regiões em um único dataframe para cada ano e armazenando na camada Prata

docentes_co_2017_silver.write.format('delta').mode('overwrite').save(blobPath + silverPath + 'docentes_2017')
docentes_norte_2017_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes_2017')
docentes_nordeste_2017_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes_2017')
docentes_sul_2017_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes_2017')
docentes_sudeste_2017_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes_2017')

docentes_co_2018_silver.write.format('delta').mode('overwrite').save(blobPath + silverPath + 'docentes_2018')
docentes_norte_2018_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes_2018')
docentes_nordeste_2018_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes_2018')
docentes_sul_2018_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes_2018')
docentes_sudeste_2018_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes_2018')

docentes_co_2019_silver.write.format('delta').mode('overwrite').save(blobPath + silverPath + 'docentes_2019')
docentes_norte_2019_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes_2019')
docentes_nordeste_2019_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes_2019')
docentes_sul_2019_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes_2019')
docentes_sudeste_2019_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes_2019')

In [0]:
# Leitura dos dataframes unificados

docentes_2017_silver = spark.read.format("delta").load(blobPath + silverPath + 'docentes_2017')
docentes_2018_silver = spark.read.format("delta").load(blobPath + silverPath + 'docentes_2018')
docentes_2019_silver = spark.read.format("delta").load(blobPath + silverPath + 'docentes_2019')

In [0]:
# Validação da unificação dos dataframes, comparando o total de registros

totalBronzeRows = docentes_co_2017_silver.count() + docentes_norte_2017_silver.count() + docentes_nordeste_2017_silver.count() + docentes_sul_2017_silver.count() + docentes_sudeste_2017_silver.count()

print("2017:", docentes_2017_silver.count() == totalBronzeRows)

totalBronzeRows = docentes_co_2018_silver.count() + docentes_norte_2018_silver.count() + docentes_nordeste_2018_silver.count() + docentes_sul_2018_silver.count() + docentes_sudeste_2018_silver.count()

print("2018:", docentes_2018_silver.count() == totalBronzeRows)

totalBronzeRows = docentes_co_2019_silver.count() + docentes_norte_2019_silver.count() + docentes_nordeste_2019_silver.count() + docentes_sul_2019_silver.count() + docentes_sudeste_2019_silver.count()

print("2019:", docentes_2019_silver.count() == totalBronzeRows)

2017: True
2018: True
2019: True

In [0]:
# Verificando a quantidade de registros null por coluna de cada dataframe

display(docentes_2017_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in docentes_2017_silver.columns]))
display(docentes_2018_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in docentes_2018_silver.columns]))
display(docentes_2019_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in docentes_2019_silver.columns]))

Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Escolaridade,Situacao_Curso_1,Area_Curso_1,Curso_1,Licenciatura_Curso_1,Situacao_Curso_2,Area_Curso_2,Curso_2,Licenciatura_Curso_2,Situacao_Curso_3,Area_Curso_3,Curso_3,Licenciatura_Curso_3,Especializacao,Mestrado,Doutorado,Pos_Graduacao,Funcao
0,0,0,0,0,0,110778,0,1316985,1316985,1316985,1316985,11001440,11001440,11001440,11001440,11667872,11667872,11667872,11667872,1975638,1975638,1975638,1975638,0


Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Escolaridade,Situacao_Curso_1,Area_Curso_1,Curso_1,Licenciatura_Curso_1,Situacao_Curso_2,Area_Curso_2,Curso_2,Licenciatura_Curso_2,Situacao_Curso_3,Area_Curso_3,Curso_3,Licenciatura_Curso_3,Especializacao,Mestrado,Doutorado,Pos_Graduacao,Funcao
0,0,0,0,0,0,166300,0,1098441,1098441,1098441,0,10846686,10846686,10846686,0,11518079,11518079,11518079,0,1676827,1676827,1676827,1676827,0


Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Escolaridade,Situacao_Curso_1,Area_Curso_1,Curso_1,Licenciatura_Curso_1,Situacao_Curso_2,Area_Curso_2,Curso_2,Licenciatura_Curso_2,Situacao_Curso_3,Area_Curso_3,Curso_3,Licenciatura_Curso_3,Especializacao,Mestrado,Doutorado,Pos_Graduacao,Funcao
0,0,0,0,0,0,213053,0,1236090,1236090,1236090,0,10367014,10367014,10367014,0,11391541,11391541,11391541,0,1236090,1236090,1236090,1236090,0


In [0]:
#Definição de dicionário para substituição dos valores nulos no formato -> 'Coluna':'Novo Valor'

nullSubstDict = {'Zona_Residencial': '3', 
                 'Situacao_Curso_1': '0', 'Area_Curso_1': '0', 'Curso_1': '000000', 'Licenciatura_Curso_1': '2', 
                 'Situacao_Curso_2': '0', 'Area_Curso_2': '0', 'Curso_2': '000000', 'Licenciatura_Curso_2': '2', 
                 'Situacao_Curso_3': '0', 'Area_Curso_3': '0', 'Curso_3': '000000', 'Licenciatura_Curso_3': '2', 
                 'Especializacao': '3', 'Mestrado': '3', 'Doutorado': '3', 'Pos_Graduacao': '3'}

In [0]:
# Substituição dos valores nulos de cada dataframe com base no dicionário definido

docentes_2017_silver = docentes_2017_silver.fillna(nullSubstDict)
docentes_2018_silver = docentes_2018_silver.fillna(nullSubstDict)
docentes_2019_silver = docentes_2019_silver.fillna(nullSubstDict)

In [0]:
# Verificando a quantidade de registros null por coluna de cada dataframe

display(docentes_2017_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in docentes_2017_silver.columns]))
display(docentes_2018_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in docentes_2018_silver.columns]))
display(docentes_2019_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in docentes_2019_silver.columns]))

Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Escolaridade,Situacao_Curso_1,Area_Curso_1,Curso_1,Licenciatura_Curso_1,Situacao_Curso_2,Area_Curso_2,Curso_2,Licenciatura_Curso_2,Situacao_Curso_3,Area_Curso_3,Curso_3,Licenciatura_Curso_3,Especializacao,Mestrado,Doutorado,Pos_Graduacao,Funcao
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Escolaridade,Situacao_Curso_1,Area_Curso_1,Curso_1,Licenciatura_Curso_1,Situacao_Curso_2,Area_Curso_2,Curso_2,Licenciatura_Curso_2,Situacao_Curso_3,Area_Curso_3,Curso_3,Licenciatura_Curso_3,Especializacao,Mestrado,Doutorado,Pos_Graduacao,Funcao
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Escolaridade,Situacao_Curso_1,Area_Curso_1,Curso_1,Licenciatura_Curso_1,Situacao_Curso_2,Area_Curso_2,Curso_2,Licenciatura_Curso_2,Situacao_Curso_3,Area_Curso_3,Curso_3,Licenciatura_Curso_3,Especializacao,Mestrado,Doutorado,Pos_Graduacao,Funcao
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Unificando os 3 dataframes em um único dataframe

docentes_2017_silver.write.format('delta').mode('overwrite').save(blobPath + silverPath + 'docentes')
docentes_2018_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes')
docentes_2019_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'docentes')

In [0]:
# Leitura do dataframe final para execução das últimas transformações

docentes_silver = spark.read.format("delta").load(blobPath + silverPath + 'docentes')

In [0]:
display(docentes_silver)

Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Escolaridade,Situacao_Curso_1,Area_Curso_1,Curso_1,Licenciatura_Curso_1,Situacao_Curso_2,Area_Curso_2,Curso_2,Licenciatura_Curso_2,Situacao_Curso_3,Area_Curso_3,Curso_3,Licenciatura_Curso_3,Especializacao,Mestrado,Doutorado,Pos_Graduacao,Funcao
2017,46,46,2,2,1,1,4,1,3,321C02,0,0,0,000000,2,0,0,000000,2,0,0,0,1,3
2017,35,36,1,3,1,1,4,1,1,146F04,1,0,0,000000,2,0,0,000000,2,0,0,0,1,1
2017,30,31,1,1,1,1,4,1,1,146F15,1,0,0,000000,2,0,0,000000,2,0,0,0,1,1
2017,57,57,2,1,1,1,4,1,2,210A01,0,0,0,000000,2,0,0,000000,2,0,0,0,1,1
2017,41,41,2,1,1,1,4,1,4,442Q01,0,0,0,000000,2,0,0,000000,2,0,0,0,1,1
2017,53,53,2,3,1,1,4,1,1,145F11,1,1,1,142P01,1,0,0,000000,2,1,0,0,0,1
2017,48,49,2,1,1,1,4,1,1,145F15,1,0,0,000000,2,0,0,000000,2,0,0,0,1,1
2017,37,37,2,1,1,1,4,1,1,145F08,1,0,0,000000,2,0,0,000000,2,0,0,0,1,1
2017,55,56,1,1,1,1,4,1,1,142P01,1,1,1,146F15,1,0,0,000000,2,1,0,0,0,1
2017,46,47,2,1,1,1,4,1,1,145F10,1,0,0,000000,2,0,0,000000,2,1,0,0,0,1


In [0]:
# Devido ao tamanho dos dados da coluna 'Curso_*' é definido duas variáveis para os valores para melhor visualização

codCursos = ['142A01', '142C01', '142P01', '144F12', '144F13', '145F01', '145F02', '145F05', '145F08', '145F09', '145F10', '145F11', '145F14', '145F15', '145F17', '145F18', '145F21', '145F24', '145F28', '146F02', '146F04', '146F05', '146F07', '146F09', '146F15', '146F20', '146F22', '146P01', '210A01', '211A02', '212C02', '212D01', '212M02', '212T01', '213A05', '213C06', '213C07', '213F01', '213P02', '213P03', '213P05', '213P07', '214D02', '214D05', '214D06', '214M01', '214P01', '215C02', '215F01', '220H01', '220L03', '221T01', '222L01', '223C01', '223L01', '223L02', '225A01', '225H01', '225M01', '225M02', '226F01', '310C02', '311P02', '312A01', '313C01', '313R01', '314E02', '321C01', '321C02', '321J01', '321R01', '322A01', '322B01', '340N02', '341N01', '342C01', '342M02', '342P02', '342R01', '343S01', '344C02', '345A01', '345A02', '345A07', '345A10', '345C01', '345G09', '345G10', '345G13', '345G16', '345G17', '345G26', '346S01', '346S03', '380D01', '421B07', '421B12', '421C01', '422S01', '440C01', '441F01', '441R01', '442Q01', '443C01', '443G03', '443G05', '443G06', '443M01', '443O01', '461M01', '462C01', '462E01', '481A01', '481B01', '481C01', '481T01', '481T02', '482U01', '483S01', '483S02', '520A01', '520E01', '520E04', '520E05', '520E09', '520G01', '520M01', '520P02', '520T01', '521E05', '521E06', '521M03', '521T02', '521T03', '522D02', '522E06', '522E08', '522R01', '522T02', '523B01', '523E04', '523E09', '523E10', '523E11', '523E12', '523M01', '523S03', '523T01', '523T04', '523T05', '523T06', '524E01', '524E06', '524E07', '524T03', '524T04', '525A01', '525C04', '525E04', '525E05', '525E08', '525M01', '525S01', '540F02', '540F03', '541E01', '541I02', '541P05', '541P09', '541T01', '541T02', '541T03', '542B01', '542E03', '542I01', '542I02', '543C01', '543F03', '543F05', '543P06', '544E01', '544E05', '544E07', '544M02', '544R01', '544T01', '581A05', '582A01', '582A02', '582C05', '582E02', '582E03', '582M02', '582O01', '582T04', '621A03', '621A04', '621A06', '621E03', '621M02', '621T01', '621T03', '621T04', '621T05', '621Z01', '622H01', '623E01', '623S01', '624A01', '624E01', '624T01', '641M01', '720E01', '720N01', '720S01', '721M01', '721O02', '723E01', '724O01', '725T06', '726F01', '726F03', '726N02', '726O01', '726Q01', '726T01', '727F01', '762S01', '811G01', '811H02', '811H03', '812E01', '812P01', '812T01', '813F02', '813G02', '814E02', '815E01', '840A01', '840C04', '840C05', '840N02', '840S01', '840S02', '840T02', '850G01', '861S02', '861S03', '862S01', '863C01', '863C02', '863F01', '999990', '999991', '999992', '000000']

nomeCursos = ['Processos Escolares - Tecnológico', 'Pedagogia (Ciências da Educação) - Bacharelado', 'Pedagogia - Licenciatura', 'Licenciatura Interdisciplinar em Ciências Humanas - Licenciatura', 'Licenciatura Intercultural Indígena - Licenciatura', 'Ciências Biológicas - Licenciatura', 'Ciências Naturais - Licenciatura', 'Educação Religiosa - Licenciatura', 'Filosofia - Licenciatura', 'Física - Licenciatura', 'Geografia - Licenciatura', 'História - Licenciatura', 'Letras - Língua Estrangeira - Licenciatura', 'Letras - Língua Portuguesa - Licenciatura', 'Letras - Língua Portuguesa e Estrangeira - Licenciatura', 'Matemática - Licenciatura', 'Química - Licenciatura', 'Ciências Sociais - Licenciatura', 'Libras - Licenciatura', 'Licenciatura Interdisciplinar em Artes (Educação Artística) - Licenciatura', 'Artes Visuais - Licenciatura', 'Informática - Licenciatura', 'Dança - Licenciatura', 'Licenciatura Interdisciplinar em Educação no Campo - Licenciatura', 'Educação Física - Licenciatura', 'Música - Licenciatura', 'Teatro - Licenciatura', 'Licenciatura para a Educação Profissional e Tecnológica - Licenciatura', 'Bacharelado Interdisciplinar em Artes - Bacharelado', 'Artes Visuais - Bacharelado', 'Produção cênica - Tecnológico', 'Dança - Bacharelado', 'Música - Bacharelado', 'Teatro - Bacharelado', 'Produção audiovisual - Tecnológico', 'Design gráfico - Tecnológico', 'Carnaval - Tecnológico', 'Fotografia - Tecnológico', 'Produção multimídia - Tecnológico', 'Produção fonográfica - Tecnológico', 'Produção publicitária - Tecnológico', 'Produção Cultural - Tecnológico', 'Design de moda - Tecnológico', 'Design - Bacharelado', 'Design de Interiores  - Tecnológico', 'Moda - Bacharelado', 'Design de produto - Tecnológico', 'Conservação e restauro - Tecnológico', 'Fabricação de Instrumentos Musicais - Tecnológico', 'Bacharelado Interdisciplinar Ciências Humanas - Bacharelado', 'Letras - Língua Portuguesa e Estrangeira - Bacharelado', 'Teologia - Bacharelado', 'Letras - Língua Estrangeira - Bacharelado', 'Comunicação assistiva - Tecnológico', 'Letras - Língua Portuguesa - Bacharelado', 'Libras - Bacharelado', 'Arqueologia - Bacharelado', 'História - Bacharelado', 'Museologia - Bacharelado', 'Museografia - Tecnológico', 'Filosofia - Bacharelado', 'Ciências Sociais - Bacharelado', 'Psicologia - Bacharelado', 'Antropologia - Bacharelado', 'Ciência política - Bacharelado', 'Relações Internacionais - Bacharelado', 'Ciências Econômicas - Bacharelado', 'Cinema e Audiovisual - Bacharelado', 'Comunicação Social (Área Geral) - Bacharelado', 'Jornalismo - Bacharelado', 'Radio, TV, Internet - Bacharelado', 'Arquivologia - Bacharelado', 'Biblioteconomia - Bacharelado', 'Comércio exterior - Tecnológico', 'Negócios imobiliários - Tecnológico', 'Comunicação institucional - Tecnológico', 'Marketing - Tecnológico', 'Publicidade e Propaganda - Bacharelado', 'Relações Públicas - Bacharelado', 'Gestão de Seguros - Teconológico', 'Ciências Contábeis - Bacharelado', 'Administração - Bacharelado', 'Gestão de cooperativas  - Tecnológico', 'Gestão hospitalar  - Tecnológico', 'Gestão pública  - Tecnológico', 'Processos gerenciais  - Tecnológico', 'Gestão de recursos humanos  - Tecnológico', 'Gestão da qualidade  - Tecnológico', 'Logística  - Tecnológico', 'Gestão comercial  - Tecnológico', 'Gestão financeira  - Tecnológico', 'Gestão de segurança privada - Tecnológico', 'Secretariado  - Tecnológico', 'Secretariado Executivo - Bacharelado', 'Direito - Bacharelado', 'Biomedicina - Bacharelado', 'Biotecnologia - Tecnológico', 'Ciências Biológicas - Bacharelado', 'Saneamento ambiental - Tecnológico', 'Bacharelado Interdisciplinar em Ciência e Tecnologia  - Bacharelado', 'Física - Bacharelado', 'Física Medica e Radioterapia - Bacharelado', 'Química - Bacharelado', 'Ciência da Terra - Licenciatura', 'Geofísica - Bacharelado', 'Geografia - Bacharelado', 'Geologia - Bacharelado', 'Meteorologia - Bacharelado', 'Oceanografia - Bacharelado', 'Matemática - Bacharelado', 'Ciências Atuariais - Bacharelado', 'Estatística - Bacharelado', 'Redes de computadores - Tecnológico', 'Banco de dados - Tecnológico', 'Ciência da Computação - Bacharelado', 'Gestão da tecnologia da informação - Tecnológico', 'Jogos Digitais - Tecnológico', 'Sistemas para internet - Tecnológico', 'Análise e desenvolvimento de sistemas / Segurança da informação - Tecnológico', 'Sistemas de Informação - Bacharelado', 'Automação industrial - Tecnológico', 'Engenharia - Bacharelado', 'Engenharia de Materiais - Bacharelado', 'Engenharia de Produção - Bacharelado', 'Engenharia Ambiental e Sanitária - Bacharelado', 'Geoprocessamento - Tecnológico', 'Manutenção industrial - Tecnológico', 'Gestão da produção industrial - Tecnológico', 'Gestão de telecomunicações - Tecnológico', 'Engenharia Mecânica - Bacharelado', 'Engenharia Metalúrgica - Bacharelado', 'Mecânica de precisão - Tecnológico', 'Processos metalúrgicos - Tecnológico', 'Fabricação mecânica - Tecnológico', 'Sistemas elétricos - Tecnológico', 'Engenharia Elétrica - Bacharelado', 'Sistemas de energia - Tecnológico', 'Refrigeração/Aquecimento - Tecnológico', 'Eletrotécnica industrial - Tecnológico', 'Engenharia Biomédica - Bacharelado', 'Engenharia de Computação - Bacharelado', 'Engenharia Eletrônica - Bacharelado', 'Engenharia mecatrônica - Bacharelado', 'Engenharia de Controle e Automação - Bacharelado', 'Engenharia de Telecomunicações - Bacharelado', 'Sistemas biomédicos - Tecnológico', 'Sistemas eletrônicos - Tecnológico', 'Redes de telecomunicações / Sistemas de telecomunicações - Tecnológico', 'Mecatrônica industrial - Tecnológico', 'Telemática - Tecnológico', 'Eletrônica industrial - Tecnológico', 'Engenharia de Bioprocessos - Bacharelado', 'Engenharia nuclear - Bacharelado', 'Engenharia Química - Bacharelado', 'Processos químicos - Tecnológico', 'Biocombustíveis - Tecnológico', 'Mecanização Agrícola - Tecnológico', 'Construção naval - Tecnológico', 'Engenharia Aeronáutica - Bacharelado', 'Engenharia automotiva - Bacharelado', 'Engenharia Naval - Bacharelado', 'Manutenção de aeronaves - Tecnológico', 'Sistemas Automotivos - Tecnológico', 'Produção Joalheira/Design de jóias e gemas - Tecnológico', 'Produção Gráfica  - Tecnológico', 'Engenharia de Alimentos - Bacharelado', 'Laticínios  - Tecnológico', 'Processamento de Carnes  - Tecnológico', 'Viticultura e Enologia  - Tecnológico', 'Alimentos  - Tecnológico', 'Produção Sucroalcooleira  - Tecnológico', 'Produção de Cachaça  - Tecnológico', 'Bioenergia - Tecnológico', 'Engenharia Têxtil - Bacharelado', 'Produção de vestuário  - Tecnológico', 'Produção têxtil  - Tecnológico', 'Cerâmica - Tecnológico', 'Produção moveleira  - Tecnológico', 'Papel e celulose  - Tecnológico', 'Polímeros - Tecnológico', 'Engenharia de Minas - Bacharelado', 'Petróleo e gás  - Tecnológico', 'Engenharia de Petróleo - Bacharelado', 'Mineração e extração - Tecnológico', 'Rochas ornamentais  - Tecnológico', 'Tecnologia de Mineração - Tecnológico', 'Arquitetura e Urbanismo - Bacharelado', 'Obras hidráulicas - Tecnológico', 'Agrimensura - Tecnológico', 'Construção de Edifícios - Tecnológico', 'Engenharia Cartográfica e de Agrimensura - Bacharelado', 'Engenharia Civil - Bacharelado', 'Material de construção - Tecnológico', 'Controle de obras - Tecnológico', 'Estradas - Tecnológico', 'Agroindústria - Tecnológico', 'Agronomia - Bacharelado', 'Agroecologia - Tecnológico', 'Engenharia Agrícola - Bacharelado', 'Produção Agrícola - Tecnológico', 'Irrigação e Drenagem - Tecnológico', 'Agronegócio - Tecnológico', 'Cafeicultura - Tecnológico', 'Produção de Grãos - Tecnológico', 'Zootecnia - Bacharelado', 'Horticultura - Tecnológico', 'Engenharia Florestal - Bacharelado', 'Silvicultura - Tecnológico', 'Aqüicultura - Tecnológico', 'Engenharia de Pesca - Bacharelado', 'Produção Pesqueira - Tecnológico', 'Medicina Veterinária - Bacharelado', 'Educação Física - Bacharelado', 'Naturologia - Bacharelado', 'Bacharelado Interdisciplinar Ciências da Saúde - Bacharelado', 'Medicina - Bacharelado', 'Oftálmica - Tecnológico', 'Enfermagem - Bacharelado', 'Odontologia - Bacharelado', 'Radiologia - Tecnológico', 'Fisioterapia - Bacharelado', 'Fonoaudiologia - Bacharelado', 'Nutrição - Bacharelado', 'Óptica e Optometria - Tecnológico', 'Quiropraxia - Bacharelado', 'Terapia Ocupacional - Bacharelado', 'Farmácia - Bacharelado', 'Serviço Social - Bacharelado', 'Gastronomia - Tecnológico', 'Hotelaria - Tecnológico', 'Hotelaria Hospitalar - Tecnológico', 'Eventos - Tecnológico', 'Gestão de Turismo - Tecnológico', 'Turismo - Bacharelado', 'Futebol - Tecnológico', 'Gestão desportiva e de lazer - Tecnológico', 'Economia doméstica - Bacharelado', 'Estética e Cosmética - Tecnológico', 'Pilotagem profissional de aeronaves - Tecnológico', 'Ciências Aeronáuticas - Bacharelado', 'Ciências Navais - Bacharelado', 'Sistemas de navegação fluvial - Tecnológico', 'Gestão portuária - Tecnológico', 'Transporte aéreo - Tecnológico', 'Transporte terrestre - Tecnológico', 'Processos ambientais / Gestão ambiental - Tecnológico', 'Segurança no trânsito / Segurança pública - Tecnológico', 'Serviços penais - Tecnológico', 'Segurança no trabalho - Tecnológico', 'Ciências Militares - Bacharelado', 'Ciências da Logística - Bacharelado', 'Formação Militar - Bacharelado', 'Outro curso de formação superior - Licenciatura', 'Outro curso de formação superior - Bacharelado', 'Outro curso de formação superior - Tecnológico', 'Não Informado']

In [0]:
# Alteração dos valores de cada coluna de acordo com as informações correspondentes

docentes_silver = docentes_silver.replace(['1', '2'], 
                                            ['Masculino', 'Feminino'], 'Sexo')

docentes_silver = docentes_silver.replace(['0', '1', '2', '3', '4', '5'], 
                                            ['Não Declarada', 'Branca', 'Preta', 'Parda', 'Amarela', 'Indígena'], 'Cor_Raca')

docentes_silver = docentes_silver.replace(['1', '2', '3'], 
                                            ['Brasileira', 'Brasileira - nascido no exterior ou naturalizado', 'Estrangeira'], 'Nacionalidade')

docentes_silver = docentes_silver.replace(['0', '1', '2', '3'], 
                                            ['Não Aplicável', 'Urbana', 'Rural', 'Não Informado'], 'Zona_Residencial')

docentes_silver = docentes_silver.replace(['1', '2', '3', '4'], 
                                          ['Ensino Fundamental Incompleto', 'Ensino Fundamental Completo', 'Ensino Médio Completo', 'Ensino Superior Completo'],
                                          'Escolaridade')

docentes_silver = docentes_silver.replace(['0', '1', '2'], 
                                          ['Não Informado', 'Concluído', 'Em andamento'], 'Situacao_Curso_1')

docentes_silver = docentes_silver.replace(['1', '2', '3', '4', '5', '6', '7', '8', '9'], 
                                          ['Educação', 'Humanidades e Artes', 'Ciências sociais, negócios e direitos', 'Ciências, Matemática e Computação', 
                                           'Engenharia, Produção e Construção', 'Agricultura e Veterinária', 'Saúde e Bem-estar Social', 'Serviços', 'Outros'], 
                                          'Area_Curso_1')

docentes_silver = docentes_silver.replace(codCursos, nomeCursos, 'Curso_1')

docentes_silver = docentes_silver.replace(['0', '1', '2'], ['Não', 'Sim', 'Não Informado'], 'Licenciatura_Curso_1')

docentes_silver = docentes_silver.replace(['0', '1', '2'], 
                                          ['Não Informado', 'Concluído', 'Em andamento'], 'Situacao_Curso_2')

docentes_silver = docentes_silver.replace(['1', '2', '3', '4', '5', '6', '7', '8', '9'], 
                                          ['Educação', 'Humanidades e Artes', 'Ciências sociais, negócios e direitos', 'Ciências, Matemática e Computação', 
                                           'Engenharia, Produção e Construção', 'Agricultura e Veterinária', 'Saúde e Bem-estar Social', 'Serviços', 'Outros'], 
                                          'Area_Curso_2')

docentes_silver = docentes_silver.replace(codCursos, nomeCursos, 'Curso_2')

docentes_silver = docentes_silver.replace(['0', '1', '2'], ['Não', 'Sim', 'Não Informado'], 'Licenciatura_Curso_2')

docentes_silver = docentes_silver.replace(['0', '1', '2'], 
                                          ['Não Informado', 'Concluído', 'Em andamento'], 'Situacao_Curso_3')

docentes_silver = docentes_silver.replace(['1', '2', '3', '4', '5', '6', '7', '8', '9'], 
                                          ['Educação', 'Humanidades e Artes', 'Ciências sociais, negócios e direitos', 'Ciências, Matemática e Computação', 
                                           'Engenharia, Produção e Construção', 'Agricultura e Veterinária', 'Saúde e Bem-estar Social', 'Serviços', 'Outros'], 
                                          'Area_Curso_3')

docentes_silver = docentes_silver.replace(codCursos, nomeCursos, 'Curso_3')

docentes_silver = docentes_silver.replace(['0', '1', '2'], ['Não', 'Sim', 'Não Informado'], 'Licenciatura_Curso_3')

docentes_silver = docentes_silver.replace(['0', '1', '2', '3'], 
                                          ['Não', 'Sim', 'Não Aplicável (Curso Superior Não Concluído)', 'Não Informado'], 'Especializacao')

docentes_silver = docentes_silver.replace(['0', '1', '2', '3'], 
                                          ['Não', 'Sim', 'Não Aplicável (Curso Superior Não Concluído)', 'Não Informado'], 'Mestrado')

docentes_silver = docentes_silver.replace(['0', '1', '2', '3'], 
                                          ['Não', 'Sim', 'Não Aplicável (Curso Superior Não Concluído)', 'Não Informado'], 'Doutorado')

docentes_silver = docentes_silver.replace(['0', '1', '2', '3'], 
                                          ['Não', 'Sim', 'Não Aplicável (Curso Superior Não Concluído)', 'Não Informado'], 'Pos_Graduacao')

docentes_silver = docentes_silver.replace(['1', '2', '3', '4', '5', '6', '7', '8'], 
                                          ['Docente', 'Auxiliar/Assistente Educacional', 'Profissional/Monitor de atividade complementar', 
                                           'Tradutor Intérprete de Libras', 'Docente Titular - Coordenador de Tutoria EAD', 'Docente Tutor - Auxiliar EAD', 
                                           'Guia intérprete', 'Profissional de Apoio Escolar para Alunos com Deficiência'], 'Funcao')

In [0]:
# Visualização do dataframe após alteração

display(docentes_silver)

Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Escolaridade,Situacao_Curso_1,Area_Curso_1,Curso_1,Licenciatura_Curso_1,Situacao_Curso_2,Area_Curso_2,Curso_2,Licenciatura_Curso_2,Situacao_Curso_3,Area_Curso_3,Curso_3,Licenciatura_Curso_3,Especializacao,Mestrado,Doutorado,Pos_Graduacao,Funcao
2017,46,46,Feminino,Preta,Brasileira,Urbana,Ensino Superior Completo,Concluído,"Ciências sociais, negócios e direitos",Comunicação Social (Área Geral) - Bacharelado,Não,Não Informado,0,Não Informado,Não Informado,Não Informado,0,Não Informado,Não Informado,Não,Não,Não,Sim,Profissional/Monitor de atividade complementar
2017,35,36,Masculino,Parda,Brasileira,Urbana,Ensino Superior Completo,Concluído,Educação,Artes Visuais - Licenciatura,Sim,Não Informado,0,Não Informado,Não Informado,Não Informado,0,Não Informado,Não Informado,Não,Não,Não,Sim,Docente
2017,30,31,Masculino,Branca,Brasileira,Urbana,Ensino Superior Completo,Concluído,Educação,Educação Física - Licenciatura,Sim,Não Informado,0,Não Informado,Não Informado,Não Informado,0,Não Informado,Não Informado,Não,Não,Não,Sim,Docente
2017,57,57,Feminino,Branca,Brasileira,Urbana,Ensino Superior Completo,Concluído,Humanidades e Artes,Bacharelado Interdisciplinar em Artes - Bacharelado,Não,Não Informado,0,Não Informado,Não Informado,Não Informado,0,Não Informado,Não Informado,Não,Não,Não,Sim,Docente
2017,41,41,Feminino,Branca,Brasileira,Urbana,Ensino Superior Completo,Concluído,"Ciências, Matemática e Computação",Química - Bacharelado,Não,Não Informado,0,Não Informado,Não Informado,Não Informado,0,Não Informado,Não Informado,Não,Não,Não,Sim,Docente
2017,53,53,Feminino,Parda,Brasileira,Urbana,Ensino Superior Completo,Concluído,Educação,História - Licenciatura,Sim,Concluído,Educação,Pedagogia - Licenciatura,Sim,Não Informado,0,Não Informado,Não Informado,Sim,Não,Não,Não,Docente
2017,48,49,Feminino,Branca,Brasileira,Urbana,Ensino Superior Completo,Concluído,Educação,Letras - Língua Portuguesa - Licenciatura,Sim,Não Informado,0,Não Informado,Não Informado,Não Informado,0,Não Informado,Não Informado,Não,Não,Não,Sim,Docente
2017,37,37,Feminino,Branca,Brasileira,Urbana,Ensino Superior Completo,Concluído,Educação,Filosofia - Licenciatura,Sim,Não Informado,0,Não Informado,Não Informado,Não Informado,0,Não Informado,Não Informado,Não,Não,Não,Sim,Docente
2017,55,56,Masculino,Branca,Brasileira,Urbana,Ensino Superior Completo,Concluído,Educação,Pedagogia - Licenciatura,Sim,Concluído,Educação,Educação Física - Licenciatura,Sim,Não Informado,0,Não Informado,Não Informado,Sim,Não,Não,Não,Docente
2017,46,47,Feminino,Branca,Brasileira,Urbana,Ensino Superior Completo,Concluído,Educação,Geografia - Licenciatura,Sim,Não Informado,0,Não Informado,Não Informado,Não Informado,0,Não Informado,Não Informado,Sim,Não,Não,Não,Docente


In [0]:
docentes_silver.write.format('delta').mode('overwrite').save(blobPath + goldPath + 'docentes')
docentes_silver.write.format('delta').mode('overwrite').saveAsTable('Ouro_Censo_Escolar_Docentes')